In [7]:
pip install swig

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install box2D

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.3 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install pygame

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 21.3 MB/s  0:00:00m0:00:010:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install stable-baselines3


  Using cached pandas-2.3.3-cp313-cp313-macosx_11_0_arm64.whl.metadata (91 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached markupsafe-3.0.3-cp313-cp313-macosx_11_0_arm64.whl.metadata (2.7 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 MB 26.1 MB/s  0:00:02 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.4 MB/s  0:00:00
Using cached sympy-1.14.0-py3-none-any.whl (6.3 MB)
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
Using cached markupsafe-3.0.3-cp313-cp313-macosx_11_0_arm64.whl (12 kB)
Using cached pandas-2.3.3-cp313-cp313-macosx_11_0_arm64.whl (10.7 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
U

In [6]:
pip install annular_driving_env

ERROR: Could not find a version that satisfies the requirement annular_driving_env (from versions: none)
ERROR: No matching distribution found for annular_driving_env
Note: you may need to restart the kernel to use updated packages.


In [9]:

# =========================
import gymnasium as gym
from gymnasium import spaces
import numpy as np
from stable_baselines3 import PPO
import time



class AnnularDrivingEnv(gym.Env):
    metadata = {"render_modes": ["human"]}

    def __init__(self):
        super().__init__()

        # ---- Track ----
        self.R_inner = 8.0
        self.R_outer = 12.0
        self.R_mid = (self.R_inner + self.R_outer) / 2
        self.track_width = self.R_outer - self.R_inner

        # ---- Vehicle ----
        self.L = 2.5
        self.v_max = 15.0
        self.a_max = 3.0
        self.delta_max = np.pi / 2
        self.dt = 0.05

        # ---- Action space ----
        self.action_space = spaces.Box(
            low=np.array([-self.a_max, -self.delta_max]),
            high=np.array([self.a_max, self.delta_max]),
            dtype=np.float32
        )

        # ---- Observation space ----
        self.observation_space = spaces.Box(
            low=-1.0,
            high=1.0,
            shape=(6,),
            dtype=np.float32
        )

        self.reset()

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)

        self.r = self.R_mid
        self.phi = 0.0
        self.theta = np.pi / 2
        self.v = 0.0
        self.prev_phi = self.phi

        return self._get_obs(), {}

    def _get_obs(self):
        return np.array([
            (self.r - self.R_mid) / self.track_width,
            np.sin(self.phi),
            np.cos(self.phi),
            np.sin(self.theta),
            np.cos(self.theta),
            self.v / self.v_max
        ], dtype=np.float32)

    def step(self, action):
        a, delta = action
        delta = np.clip(delta, -self.delta_max, self.delta_max)

        # --- Kinematic bicycle model ---
        self.v = np.clip(self.v + a * self.dt, 0, self.v_max)
        self.theta += (self.v / self.L) * np.tan(delta) * self.dt

        dr = self.v * np.sin(self.theta - self.phi) * self.dt
        dphi = (self.v / max(self.r, 1e-3)) * np.cos(self.theta - self.phi) * self.dt

        self.r += dr
        self.phi += dphi

        # --- Reward ---
        delta_phi = self.phi - self.prev_phi
        self.prev_phi = self.phi

        reward = (
            5.0 * delta_phi
            - 2.0 * abs(self.r - self.R_mid)
            - 0.1 * delta**2
        )

        terminated = False
        if self.r < self.R_inner or self.r > self.R_outer:
            reward = -100.0
            terminated = True

        return self._get_obs(), reward, terminated, False, {}

    def render(self):
        print(f"r={self.r:.2f}, phi={self.phi:.2f}, v={self.v:.2f}")



env = AnnularDrivingEnv()

model = PPO(
    "MlpPolicy",
    env,
    learning_rate=3e-4,
    gamma=0.99,
    n_steps=2048,
    batch_size=64,
    clip_range=0.2,
    verbose=1
)

model.learn(total_timesteps=300_000)
model.save("ppo_annular_driver")


obs, _ = env.reset()

for _ in range(1500):
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, _, _ = env.step(action)
    env.render()
    time.sleep(0.02)

    if done:
        obs, _ = env.reset()


/Users/dantone/Downloads/wids_rl/.venv/lib/python3.13/site-packages/gymnasium/spaces/box.py:236: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
/Users/dantone/Downloads/wids_rl/.venv/lib/python3.13/site-packages/gymnasium/spaces/box.py:306: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 465      |
|    ep_rew_mean     | -597     |
| time/              |          |
|    fps             | 6514     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 465         |
|    ep_rew_mean          | -597        |
| time/                   |             |
|    fps                  | 3729        |
|    iterations           | 2           |
|    time_elapsed         | 1           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.005476393 |
|    clip_fraction        | 0.033       |
|    clip_range           | 0.2         |
|    entropy_loss   